# Lux AI Season 1 Python Tutorial Notebook

Welcome to Lux AI Season 1!

This notebook is the basic setup to use Jupyter Notebooks and the `kaggle-environments` package to develop your bot. If you plan to not use Jupyter Notebooks or any other programming language, please see our [Github](https://github.com/Lux-AI-Challenge/Lux-Design-2021). The following are some important links!

- Competition Page: https://www.kaggle.com/c/lux-ai-2021/

- Online Visualizer: https://2021vis.lux-ai.org/

- Specifications: https://www.lux-ai.org/specs-2021

- Github: https://github.com/Lux-AI-Challenge/Lux-Design-2021

- Bot API: https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

And if you haven't done so already, we **highly recommend** you join our Discord server at https://discord.gg/aWJt3UAcgn or at the minimum follow the kaggle forums at https://www.kaggle.com/c/lux-ai-2021/discussion. We post important announcements there such as changes to rules, events, and opportunities from our sponsors!

Now let's get started!

## Prerequisites

We assume that you have a basic knowledge of Python and programming. It's okay if you don't know the game specifications yet! Feel free to always refer back to https://www.lux-ai.org/specs-2021.

## Basic Setup

First thing to verify is that you have **Node.js v12 or above**. The engine for the competition runs on Node.js (for many good reasons including an awesome visualizer) and thus it is required. You can download it [here](https://nodejs.org/en/download/). You can then verify you have the appropriate version by running


In [16]:
!node --version

v16.10.0


We will also need Kaggle Environments

In [17]:
!pip install kaggle-environments -U

Requirement already up-to-date: kaggle-environments in /home/andrea/.local/lib/python3.8/site-packages (1.8.12)


Next, we have to import the `make` function from the `kaggle_environments` package

In [18]:
from kaggle_environments import make
from random import randint

The `make` function is used to create environments that can then run the game given agents. Agents refer to programmed bots that play the game given observations of the game itself. 

In addition to making the environment, you may also pass in special configurations such as the number of episode steps (capped at 361) and the seed.

Now lets create our environment using `make` and watch a Episode! (We will be using the seed 562124210 because it's fun)

In [19]:
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
from random import randint
import math
import sys

In [31]:
### Define helper functions

# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles, unit_positions):
    unit_positions_without_pos = unit_positions
    unit_positions_without_pos.remove(pos)
    for resource_tile in resource_tiles:
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and player.researched_uranium():
            return find_closest_uranium(pos, resource_tiles, unit_positions_without_pos)
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and player.researched_coal():
            return find_closest_coal(pos, resource_tiles, unit_positions_without_pos)
        return find_closest_wood(pos, resource_tiles, unit_positions_without_pos)

def find_closest_coal(pos, resource_tiles, unit_positions):
    closest_dist = math.inf
    closest_coal_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type != Constants.RESOURCE_TYPES.COAL:
            continue
        dist = resource_tile.pos.distance_to(pos)
        move_direction = pos.direction_to(resource_tile.pos)
        unit_new_pos = pos.translate(move_direction, 1)
        if dist < closest_dist and resource_tile.pos not in unit_positions and unit_new_pos not in unit_positions:
            closest_dist = dist
            closest_coal_tile = resource_tile
    unit_positions.append(unit_new_pos)
    return (closest_coal_tile, unit_positions)

def find_closest_uranium(pos, resource_tiles, unit_positions):
    closest_dist = math.inf
    closest_uranium_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type != Constants.RESOURCE_TYPES.URANIUM:
            continue
        dist = resource_tile.pos.distance_to(pos)
        move_direction = pos.direction_to(resource_tile.pos)
        unit_new_pos = pos.translate(move_direction, 1)
        if dist < closest_dist and resource_tile.pos not in unit_positions and unit_new_pos not in unit_positions:
            closest_dist = dist
            closest_uranium_tile = resource_tile
    unit_positions.append(unit_new_pos)
    return (closest_uranium_tile, unit_positions)

def find_closest_wood(pos, resource_tiles, unit_positions):
    closest_dist = math.inf
    closest_wood_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type != Constants.RESOURCE_TYPES.WOOD:
            continue
        dist = resource_tile.pos.distance_to(pos)
        move_direction = pos.direction_to(resource_tile.pos)
        unit_new_pos = pos.translate(move_direction, 1)
        if dist < closest_dist and resource_tile.pos not in unit_positions and unit_new_pos not in unit_positions:
            closest_dist = dist
            closest_wood_tile = resource_tile
    unit_positions.append(unit_new_pos)
    return (closest_wood_tile, unit_positions)

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

# returns boolean
def should_build_another_citytile(player):
    total_citytiles = 0
    for city in player.cities.values():
        total_citytiles += len(city.citytiles)
    if len(player.units) == total_citytiles:
        return True
    
    for city in player.cities.values():
        if city.fuel <= city.get_light_upkeep() * 10:
            return False

    return True

def get_city_with_least_fuel(cities):
    lowest_fuel = math.inf
    lowest_fuel_city = None
    for city in cities.values():
        if city.fuel < lowest_fuel:
            lowest_fuel = city.fuel
            lowest_fuel_city = city
    return (lowest_fuel_city, lowest_fuel)

def pick_random_direction():
    meh = randint(0, 4)
    if meh == 0:
        return Constants.DIRECTIONS.CENTER
    elif meh == 1:
        return Constants.DIRECTIONS.EAST
    elif meh == 2:
        return Constants.DIRECTIONS.NORTH
    elif meh == 3:
        return Constants.DIRECTIONS.SOUTH
    elif meh == 4:
        return Constants.DIRECTIONS.WEST

def city_can_build_cart_or_worker(player):
    total_citytiles = 0
    for c in player.cities.values():
        total_citytiles += len(c.citytiles)
    if len(player.units) < total_citytiles:
        return True
    return False

In [34]:
game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)

    unit_positions = []
    for unit in player.units:
        unit_positions.append(unit.pos)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            # we want to mine only if there is space left in the worker's cargo
            if unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                closest_resource_tile, unit_positions = find_closest_resources(unit.pos, player, resource_tiles, unit_positions)
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    move_direction = unit.pos.direction_to(closest_resource_tile.pos)
                    action = unit.move(move_direction)
                    unit_positions.append(unit.pos.translate(move_direction, 1))
                    actions.append(action)
                else:
                    closest_city_tile = find_closest_city_tile(unit.pos, player)
                    move_direction = unit.pos.direction_to(closest_city_tile.pos)
                    action = unit.move(move_direction)
                    unit_positions.append(unit.pos.translate(move_direction, 1))
                    actions.append(action)
            else:
                lowest_fuel_city, lowest_fuel = get_city_with_least_fuel(player.cities)
                if lowest_fuel <= 230:
                    action = unit.move(unit.pos.direction_to(lowest_fuel_city.citytiles[0].pos))
                    actions.append(action)
                elif unit.pos.distance_to(lowest_fuel_city.citytiles[0].pos) > 0.25 * game_state.map_height:
                    if unit.can_build(game_state.map):
                        actions.append(unit.build_city())
                    else:
                        actions.append(unit.move(pick_random_direction()))
                elif should_build_another_citytile(player):
                    if unit.can_build(game_state.map):
                        actions.append(unit.build_city())
                    else:
                        actions.append(unit.move(pick_random_direction()))
                else:
                    # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                    closest_city_tile = find_closest_city_tile(unit.pos, player)
                    if closest_city_tile is not None:
                        # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                        move_direction = unit.pos.direction_to(closest_city_tile.pos)
                        action = unit.move(move_direction)
                        unit_positions.append(unit.pos.translate(move_direction, 1))
                        actions.append(action)
        elif unit.is_cart() and unit.can_act():
            if unit.get_cargo_space_left() > 20:
                closest_resource_tile, unit_positions = find_closest_resources(unit.pos, player, resource_tiles, unit_positions)
                if closest_resource_tile is not None:
                    move_direction = unit.pos.direction_to(closest_resource_tile.pos)
                    action = unit.move(move_direction)
                    unit_positions.append(unit.pos.translate(move_direction, 1))
                    actions.append(action)
            else:
                closest_city_tile = find_closest_city_tile(unit.pos, player)
                if closest_city_tile is not None:
                    move_direction = unit.pos.direction_to(closest_city_tile.pos)
                    actions.append(unit.move(move_direction))
                    unit_positions.append(unit.pos.translate(move_direction, 1))
    
    for city in player.cities.values():
        for city_tile in city.citytiles:
            if city_tile.can_act():
                if city_can_build_cart_or_worker(player):
                    actions.append(city_tile.build_worker())
                else:
                    actions.append(city_tile.research())

    return actions

In [36]:
env = make("lux_ai_2021", configuration={"seed": 56212210, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])
env.render(mode="ipython", width=1200, height=800)

[WARN] (match_Jr6C0femY7UZ) - turn 49; Unit u_3 collided when trying to move n to (4, 12)
[WARN] (match_Jr6C0femY7UZ) - Agent 0 tried to build unit on tile (7, 11) but unit cap reached. Build more CityTiles!; turn 66; cmd: bw 7 11
[WARN] (match_Jr6C0femY7UZ) - Agent 0 tried to build unit on tile (5, 11) but unit cap reached. Build more CityTiles!; turn 88; cmd: bw 5 11
[WARN] (match_Jr6C0femY7UZ) - Agent 0 tried to build unit on tile (8, 12) but unit cap reached. Build more CityTiles!; turn 94; cmd: bw 8 12
[WARN] (match_Jr6C0femY7UZ) - turn 99; Unit u_3 collided when trying to move n to (6, 11)
[WARN] (match_Jr6C0femY7UZ) - turn 101; Unit u_3 collided when trying to move c to (6, 12)
[WARN] (match_Jr6C0femY7UZ) - turn 101; Unit u_3 collided when trying to move c to (6, 12)
[WARN] (match_Jr6C0femY7UZ) - turn 101; Unit u_10 collided when trying to move w to (6, 12)
[WARN] (match_Jr6C0femY7UZ) - turn 101; Unit u_12 collided when trying to move e to (6, 12)
[WARN] (match_Jr6C0femY7UZ) - t

Ok so woah, what just happened? We just ran a match, that's what :)

There's a number of quality of life features in the visualizer, which you can also find embedded on the kaggle competition page when watching replays or on the online visualizer when using replay files. 

If you find this replay viewer slow, you can also download a local copy of this replay viewer in addition to lowering the graphics quality, see https://github.com/Lux-AI-Challenge/LuxViewer2021 for instructions.

At this point, we recommend reading the [game specifications](https://www.lux-ai.org/specs-2021) a bit more to understand how to build a bot that tries to win the game.

In [39]:
%%writefile agent.py
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
from random import randint
import math
import sys

### Define helper functions

# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles, unit_positions):
    unit_positions_without_pos = unit_positions
    unit_positions_without_pos.remove(pos)
    for resource_tile in resource_tiles:
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and player.researched_uranium():
            return find_closest_uranium(pos, resource_tiles, unit_positions_without_pos)
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and player.researched_coal():
            return find_closest_coal(pos, resource_tiles, unit_positions_without_pos)
        return find_closest_wood(pos, resource_tiles, unit_positions_without_pos)

def find_closest_coal(pos, resource_tiles, unit_positions):
    closest_dist = math.inf
    closest_coal_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type != Constants.RESOURCE_TYPES.COAL:
            continue
        dist = resource_tile.pos.distance_to(pos)
        move_direction = pos.direction_to(resource_tile.pos)
        unit_new_pos = pos.translate(move_direction, 1)
        if dist < closest_dist and resource_tile.pos not in unit_positions and unit_new_pos not in unit_positions:
            closest_dist = dist
            closest_coal_tile = resource_tile
    unit_positions.append(unit_new_pos)
    return (closest_coal_tile, unit_positions)

def find_closest_uranium(pos, resource_tiles, unit_positions):
    closest_dist = math.inf
    closest_uranium_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type != Constants.RESOURCE_TYPES.URANIUM:
            continue
        dist = resource_tile.pos.distance_to(pos)
        move_direction = pos.direction_to(resource_tile.pos)
        unit_new_pos = pos.translate(move_direction, 1)
        if dist < closest_dist and resource_tile.pos not in unit_positions and unit_new_pos not in unit_positions:
            closest_dist = dist
            closest_uranium_tile = resource_tile
    unit_positions.append(unit_new_pos)
    return (closest_uranium_tile, unit_positions)

def find_closest_wood(pos, resource_tiles, unit_positions):
    closest_dist = math.inf
    closest_wood_tile = None
    for resource_tile in resource_tiles:
        if resource_tile.resource.type != Constants.RESOURCE_TYPES.WOOD:
            continue
        dist = resource_tile.pos.distance_to(pos)
        move_direction = pos.direction_to(resource_tile.pos)
        unit_new_pos = pos.translate(move_direction, 1)
        if dist < closest_dist and resource_tile.pos not in unit_positions and unit_new_pos not in unit_positions:
            closest_dist = dist
            closest_wood_tile = resource_tile
    unit_positions.append(unit_new_pos)
    return (closest_wood_tile, unit_positions)

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

# returns boolean
def should_build_another_citytile(player):
    total_citytiles = 0
    for city in player.cities.values():
        total_citytiles += len(city.citytiles)
    if len(player.units) == total_citytiles:
        return True
    
    for city in player.cities.values():
        if city.fuel <= city.get_light_upkeep() * 10:
            return False

    return True

def get_city_with_least_fuel(cities):
    lowest_fuel = math.inf
    lowest_fuel_city = None
    for city in cities.values():
        if city.fuel < lowest_fuel:
            lowest_fuel = city.fuel
            lowest_fuel_city = city
    return (lowest_fuel_city, lowest_fuel)

def pick_random_direction():
    meh = randint(0, 4)
    if meh == 0:
        return Constants.DIRECTIONS.CENTER
    elif meh == 1:
        return Constants.DIRECTIONS.EAST
    elif meh == 2:
        return Constants.DIRECTIONS.NORTH
    elif meh == 3:
        return Constants.DIRECTIONS.SOUTH
    elif meh == 4:
        return Constants.DIRECTIONS.WEST

def city_can_build_cart_or_worker(player):
    total_citytiles = 0
    for c in player.cities.values():
        total_citytiles += len(c.citytiles)
    if len(player.units) < total_citytiles:
        return True
    return False

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)

    unit_positions = []
    for unit in player.units:
        unit_positions.append(unit.pos)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            # we want to mine only if there is space left in the worker's cargo
            if unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                closest_resource_tile, unit_positions = find_closest_resources(unit.pos, player, resource_tiles, unit_positions)
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    move_direction = unit.pos.direction_to(closest_resource_tile.pos)
                    action = unit.move(move_direction)
                    unit_positions.append(unit.pos.translate(move_direction, 1))
                    actions.append(action)
                else:
                    closest_city_tile = find_closest_city_tile(unit.pos, player)
                    move_direction = unit.pos.direction_to(closest_city_tile.pos)
                    action = unit.move(move_direction)
                    unit_positions.append(unit.pos.translate(move_direction, 1))
                    actions.append(action)
            else:
                lowest_fuel_city, lowest_fuel = get_city_with_least_fuel(player.cities)
                if lowest_fuel <= 230:
                    action = unit.move(unit.pos.direction_to(lowest_fuel_city.citytiles[0].pos))
                    actions.append(action)
                elif unit.pos.distance_to(lowest_fuel_city.citytiles[0].pos) > 0.25 * game_state.map_height:
                    if unit.can_build(game_state.map):
                        actions.append(unit.build_city())
                    else:
                        actions.append(unit.move(pick_random_direction()))
                elif should_build_another_citytile(player):
                    if unit.can_build(game_state.map):
                        actions.append(unit.build_city())
                    else:
                        actions.append(unit.move(pick_random_direction()))
                else:
                    # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                    closest_city_tile = find_closest_city_tile(unit.pos, player)
                    if closest_city_tile is not None:
                        # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                        move_direction = unit.pos.direction_to(closest_city_tile.pos)
                        action = unit.move(move_direction)
                        unit_positions.append(unit.pos.translate(move_direction, 1))
                        actions.append(action)
        elif unit.is_cart() and unit.can_act():
            if unit.get_cargo_space_left() > 20:
                closest_resource_tile, unit_positions = find_closest_resources(unit.pos, player, resource_tiles, unit_positions)
                if closest_resource_tile is not None:
                    move_direction = unit.pos.direction_to(closest_resource_tile.pos)
                    action = unit.move(move_direction)
                    unit_positions.append(unit.pos.translate(move_direction, 1))
                    actions.append(action)
            else:
                closest_city_tile = find_closest_city_tile(unit.pos, player)
                if closest_city_tile is not None:
                    move_direction = unit.pos.direction_to(closest_city_tile.pos)
                    actions.append(unit.move(move_direction))
                    unit_positions.append(unit.pos.translate(move_direction, 1))
    
    for city in player.cities.values():
        for city_tile in city.citytiles:
            if city_tile.can_act():
                if city_can_build_cart_or_worker(player):
                    actions.append(city_tile.build_worker())
                else:
                    actions.append(city_tile.research())

    return actions

Overwriting agent.py


In [14]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])
env.render(mode="ipython", width=1200, height=800)

NameError: name 'agent' is not defined

## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

In [40]:
!tar -czf submission.tar.gz *

## Submit
Now open the /kaggle/working folder and find submission.tar.gz, download that file, navigate to the "MySubmissions" tab in https://www.kaggle.com/c/lux-ai-2021/ and upload your submission! It should play a validation match against itself and once it succeeds it will be automatically matched against other players' submissions. Newer submissions will be prioritized for games over older ones. Your team is limited in the number of succesful submissions per day so we highly recommend testing your bot locally before submitting.

## CLI Tool

There's a separate CLI tool that can also be used to run matches. It's recommended for Jupyter Notebook users to stick with just this notebook, and all other users including python users to follow the instructions on https://github.com/Lux-AI-Challenge/Lux-Design-2021

The other benefit however of using the CLI tool is that it generates much smaller, "stateless" replays and also lets you run a mini leaderboard on multiple bots ranked by various ranking algorithms

## Additional things to check out

Make sure you check out the Bot API at https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

This documents what you can do using the starter kit files in addition to telling you how to use the annotation debug commands that let you annotate directly on a replay (draw lines, circle things etc.)

You can also run the following below to save a episode to a JSON replay file. These are the same as what is shown on the leaderbaord and you can upload the replay files to the online replay viewer https://2021vis.lux-ai.org/


For a local (faster) version of the replay viewer, follow installation instructions here https://github.com/Lux-AI-Challenge/Lux-Viewer-2021

In [ ]:
import json
replay = env.toJSON()
with open("replay.json", "w") as f:
    json.dump(replay, f)

## Suggestions / Strategies

There are a lot of places that could be improved with the agent we have in this tutorial notebook. Here are some!

- Using the build city action to build new cities and thus build new units
- Having cities perform research each turn to unlock new resources
- Writing collision-free code that lets units move smoothly around and through each other when navigating to targets
- Mining resources near your opponent's citytiles so they have less easy access to resources
- Using carts to deliver resources from far away clusters of wood, coal, uranium to a city in need
- Sending worker units over to the opponent's roads and pillaging them to slow down their agent
- Optimizing over how much to mine out of forests before letting them regrow so you can build more cities and get sustainable fuel